# Análisis de Simulaciones de Portafolios
Este notebook analiza un archivo CSV con resultados de simulaciones financieras en 6 apartados:
1. Mejores configuraciones de portafolio
2. Correlaciones entre variables
3. Estadísticas generales
4. Riesgo vs Retorno
5. Prueba de normalidad
6. Estrategias más exitosas

In [ ]:
# Librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, shapiro

In [ ]:
# Cargar el CSV (modifica la ruta si es necesario)
file_path = "tests.csv"  # Cambia esto si es necesario
df = pd.read_csv(file_path)
df = df.tail(90)
df.head()

## 1. Mejores configuraciones de portafolio

In [ ]:
grouped = (
    df.groupby(["percentil", "prediccion", "numberStocksInPortfolio", "rlog_size","margen"])
    .agg(media_tae=("tae", "mean"), conteo=("tae", "count"))
    .sort_values(by="media_tae", ascending=False)
)
grouped.head(10)

## 2. Relación entre variables

In [ ]:
cols = ["tae", "rentabilidad_total", "comision_total", "apalancamiento", "margen", "numberStocksInPortfolio"]
corr = df[cols].corr()
corr

In [ ]:
plt.figure(figsize=(8,6))
plt.imshow(corr, cmap='coolwarm', interpolation='none')
plt.colorbar(label='Correlación')
plt.xticks(range(len(cols)), cols, rotation=45)
plt.yticks(range(len(cols)), cols)
plt.title("Matriz de Correlación entre Variables")
plt.tight_layout()
plt.show()

## 3. Estadísticas generales

In [ ]:
df[cols].describe()

## 4. Análisis riesgo-retorno

In [ ]:
rr = df.groupby(["apalancamiento", "percentil", "prediccion"]).agg(media_tae=("tae", "mean"), std_tae=("tae", "std")).reset_index()
rr["sharpe"] = rr["media_tae"] / rr["std_tae"]
rr.sort_values(by="sharpe", ascending=False).head(10)

## 5. Prueba de normalidad (Shapiro-Wilk)

In [ ]:
tae_all = df["tae"].dropna()
stat, p = shapiro(tae_all)
print(f"Estadístico: {stat:.4f}, p-valor: {p:.4f}")
if p < 0.05:
    print("→ No es distribución normal")
else:
    print("→ Distribución normal aceptable")

In [ ]:
plt.figure()
count, bins, _ = plt.hist(tae_all, bins=20, density=True, alpha=0.6, edgecolor='black')
x = np.linspace(min(tae_all), max(tae_all), 100)
p_norm = norm.pdf(x, tae_all.mean(), tae_all.std())
plt.plot(x, p_norm, linewidth=2)
plt.title("Distribución TAE con Curva Normal")
plt.xlabel("TAE")
plt.ylabel("Densidad")
plt.grid()
plt.tight_layout()
plt.show()

## 6. Estrategias más exitosas

In [ ]:
df.sort_values(by="tae", ascending=False)[[
    "tae", "rentabilidad_total", "comision_total", "apalancamiento",
    "percentil", "prediccion", "numberStocksInPortfolio", "rlog_size"
]].head(10)